In [1]:
!pip install langchain langchain-community langchain-google-genai google-generativeai pdfplumber faiss-cpu tiktoken PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6

In [38]:
import os
import time
import fitz  # PyMuPDF for link extraction
import numpy as np
import pdfplumber
from typing import List, Tuple
from sklearn.preprocessing import normalize
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import google.generativeai as genai

In [2]:
GOOGLE_API_KEY = "Google API Key"
assert GOOGLE_API_KEY, "Set your GOOGLE_API_KEY as an environment variable"

In [4]:
# === Load and extract text from resume PDF ===
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text.strip()

In [19]:
# === Extract text + URL pairs from PDF ===
def extract_links_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    links = []
    for page_num in range(len(doc)):
        page = doc[page_num]
        for link in page.get_links():
            if link['uri']:  # It's an external URL
                rect = link['from']
                try:
                    text = page.get_textbox(rect)
                    if text.strip():
                        links.append((text.strip(), link['uri']))
                except:
                    continue
    return links  # list of (text, url)

In [20]:
# === Chunk text into Document objects ===
def chunk_text_with_metadata(text, links, chunk_size=800, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.create_documents([text])

    # Attach links as global metadata to each chunk
    for chunk in chunks:
        chunk.metadata["links"] = links

    return chunks

In [5]:
# === Chunking the text ===
def chunk_text(text, chunk_size=800, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.create_documents([text])

In [6]:
# === Embed documents with Gemini in batches (with rate limit) ===
def embed_and_normalize_documents(docs, model, batch_size=15, wait_time=60):
    embedded_docs = []
    all_vectors = []

    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc.page_content for doc in batch]

        print(f"Embedding batch {i // batch_size + 1} / {len(docs) // batch_size + 1}")
        try:
            embeddings = model.embed_documents(contents)
            # Normalize each embedding to unit vector (cosine similarity)
            norm_embeddings = normalize(embeddings, axis=1)

            for doc, norm_emb in zip(batch, norm_embeddings):
                all_vectors.append(norm_emb)
                embedded_docs.append(doc)

        except Exception as e:
            print(f"Error embedding batch {i}: {e}")

        if i + batch_size < len(docs):
            time.sleep(wait_time)  # Respect rate limit

    return embedded_docs, np.array(all_vectors)

In [14]:
# === Build and save FAISS vectorstore ===
def build_faiss_vectorstore(docs, vectors, embedding_model, save_path="resume_vectorstore"):
    # Create the list of tuples (text, embedding)
    text_embeddings = [(doc.page_content, vectors[i].tolist()) for i, doc in enumerate(docs)]
    faiss_store = FAISS.from_embeddings(text_embeddings=text_embeddings, embedding=embedding_model)
    faiss_store.save_local(save_path)
    print(f"✅ FAISS vectorstore saved at: {save_path}")

In [22]:
# === MAIN Pipeline ===
def process_resume(pdf_path):
    print("📄 Extracting text from PDF...")
    text = extract_text_from_pdf(pdf_path)
    #print(text)

    print("🔗 Extracting links (e.g., GitHub, LinkedIn)...")
    links = extract_links_from_pdf(pdf_path)
    for t, u in links:
        print(f"   • {t} → {u}")

    print("✂️ Chunking with metadata...")
    docs = chunk_text_with_metadata(text, links)
    #print(docs)

    print("🔗 Embedding and normalizing with Gemini...")
    # Pass the API key directly to the constructor
    model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)
    embedded_docs, norm_vectors = embed_and_normalize_documents(docs, model)
    #print(embedded_docs)
    #print(norm_vectors)

    print("💾 Building FAISS vectorstore...")
    build_faiss_vectorstore(embedded_docs, norm_vectors, model)


if __name__ == "__main__":
    process_resume("Resume_fulltime.pdf")

📄 Extracting text from PDF...
🔗 Extracting links (e.g., GitHub, LinkedIn)...
   • pavan.agni4@gmail.com → mailto:pavan.agni4@gmail.com
   • GitHub → https://github.com/PavanKAgnihotri?tab=repositories
   • LinkedIn → https://www.linkedin.com/in/pavan-k-agnihotri-026908190
✂️ Chunking with metadata...
🔗 Embedding and normalizing with Gemini...
Embedding batch 1 / 1
💾 Building FAISS vectorstore...
✅ FAISS vectorstore saved at: resume_vectorstore


Step 2: Job Description Ingestion Pipeline

In [23]:
# === Chunk the JD string ===
def chunk_job_description(jd_text, chunk_size=800, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.create_documents([jd_text])

In [25]:
# === Build and save FAISS vectorstore ===
def build_faiss_vectorstore_jd(docs, vectors, embedding_model, save_path="jd_vectorstore"):
    # Create the list of tuples (text, embedding)
    text_embeddings = [(doc.page_content, vectors[i].tolist()) for i, doc in enumerate(docs)]
    faiss_store = FAISS.from_embeddings(text_embeddings=text_embeddings, embedding=embedding_model)
    faiss_store.save_local(save_path)
    print(f"✅ FAISS vectorstore saved at: {save_path}")

In [28]:
# === MAIN ===
def process_jd_string(jd_text: str, save_path="jd_vectorstore"):
    print("✂️ Chunking JD...")
    docs_jd = chunk_job_description(jd_text)

    print("🔐 Embedding JD...")
    model_jd = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)
    embedded_docs_jd, norm_vectors_jd = embed_and_normalize_documents(docs_jd, model_jd)

    print("💾 Saving JD FAISS vectorstore...")
    build_faiss_vectorstore_jd(embedded_docs_jd, norm_vectors_jd, model_jd)

if __name__ == "__main__":
    jd_input = """
    AI Engineer
    Enterprise AI/ML Organization
    Reports to Leader of AI Engineering Group

    OVERVIEW
    We are seeking a highly skilled and forward-thinking AI Engineer to join our AI Engineering team. This role is ideal for a hands-on technologist with deep expertise in machine learning (ML) and deep learning (DL), and a strong working knowledge of Generative AI technologies. You will design, build, and optimize intelligent systems that power automation, personalization, and decision-making across our FinTech platforms. This is a unique opportunity to work at the intersection of cutting-edge AI research and real-world enterprise applications.

    RESPONSIBILITIES
    Design, develop, and deploy ML, DL, and Generative AI models that solve complex business problems and deliver measurable value.
    Build and maintain scalable ML pipelines and agentic workflows using modern frameworks and cloud-native tools.
    Collaborate with data scientists, product managers, and engineers to translate business requirements into AI-powered solutions.
    Implement and optimize AI solutions using platforms such as GCP Vertex AI, AWS Bedrock/SageMaker, and Snowflake Cortex.
    Apply techniques such as prompt engineering, RAG (Retrieval-Augmented Generation), fine-tuning, and RLHF to enhance model performance.
    Develop and deploy autonomous AI agents using frameworks like LangChain, LangGraph, and AgentSpace.
    Integrate vector databases (e.g., PGVector) and LLM orchestration tools to support retrieval and memory in generative systems.
    Ensure robust MLOps practices including CI/CD, monitoring, versioning, and lifecycle management of models.
    Stay current with AI research and industry trends, and evaluate emerging tools and techniques for enterprise adoption.
    Contribute to internal knowledge sharing, documentation, and best practices for responsible and ethical AI development.

    Must Haves:
    Bachelor’s or Master’s degree in Computer Science, Engineering, or a related field.
    4+ years of experience in AI/ML engineering, with a strong foundation in ML/DL algorithms and systems.
    Proficiency in Python and ML libraries such as TensorFlow, PyTorch, and Transformers.
    Hands-on experience with Generative AI models (e.g., GPT, Mistral, Claude) and agentic AI systems.
    Experience with NLP, conversational AI, and LLM-based applications.
    Familiarity with vector search and semantic retrieval technologies.
    Strong understanding of MLOps, including model deployment, monitoring, and retraining.
    Experience building production-grade AI systems at scale in cloud environments.
    Excellent problem-solving, communication, and collaboration skills.
    """
    process_jd_string(jd_input)

✂️ Chunking JD...
🔐 Embedding JD...
🔄 Embedding JD batch 1 / 1
💾 Saving JD FAISS vectorstore...
✅ FAISS vectorstore saved at: jd_vectorstore


Step 3: Semantic Matching Engine

In [29]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

In [36]:
# === Load vectorstores ===
def load_vectorstores(resume_path="resume_vectorstore", jd_path="jd_vectorstore"):
    print("📥 Loading vectorstores...")
    # Set allow_dangerous_deserialization to True since we trust the source (locally saved)
    resume_store = FAISS.load_local(resume_path, embedding_model, allow_dangerous_deserialization=True)
    jd_store = FAISS.load_local(jd_path, embedding_model, allow_dangerous_deserialization=True)

    resume_docs = resume_store.docstore._dict
    jd_docs = jd_store.docstore._dict

    resume_vectors = resume_store.index.reconstruct_n(0, resume_store.index.ntotal)
    jd_vectors = jd_store.index.reconstruct_n(0, jd_store.index.ntotal)

    return list(resume_docs.values()), list(jd_docs.values()), resume_vectors, jd_vectors

In [31]:
# === Compute cosine similarity matrix ===
def compute_similarity_matrix(resume_vectors, jd_vectors):
    print("🧠 Computing cosine similarity...")
    sim_matrix = np.dot(jd_vectors, resume_vectors.T)  # dot product of normalized vectors = cosine similarity
    return sim_matrix

In [33]:
# === Match JD chunks to Resume chunks ===
def rank_resume_matches(jd_docs: List[Document], resume_docs: List[Document], sim_matrix: np.ndarray, top_k=3):
    results = []

    for i, jd_doc in enumerate(jd_docs):
        jd_text = jd_doc.page_content.strip()
        sims = sim_matrix[i]
        top_indices = np.argsort(sims)[::-1][:top_k]  # descending order

        top_matches = []
        for idx in top_indices:
            match = {
                "resume_chunk": resume_docs[idx].page_content.strip(),
                "similarity": round(float(sims[idx]), 4),
                "links": resume_docs[idx].metadata.get("links", [])
            }
            top_matches.append(match)

        results.append({
            "jd_chunk": jd_text,
            "top_resume_matches": top_matches
        })

    return results

In [34]:
# === Display Matches ===
def print_matches(matches):
    for idx, result in enumerate(matches):
        print(f"\n📌 JD Chunk {idx+1}: {result['jd_chunk']}\n")
        for match in result["top_resume_matches"]:
            print(f"   🔹 Similarity: {match['similarity']}")
            print(f"   📄 Resume Match:\n   {match['resume_chunk']}")
            if match['links']:
                print(f"   🔗 Links: {match['links']}")
            print("   ---")

Step 4: Resume Rewriter + Humanizer using Gemini (LLM).

In [46]:
# === Gemini Setup ===
genai.configure(api_key=GOOGLE_API_KEY)
model_rewrite = genai.GenerativeModel("gemini-1.5-flash")

In [41]:
# === Prompt template ===
def build_prompt(jd_chunk: str, resume_chunk: str, links=None):
    links_str = ""
    if links:
        links_str = "\n\nRelevant links:\n" + "\n".join([f"{text}: {url}" for text, url in links])

    return f"""
            You are a resume rewriting assistant.

            Given the job requirement below and the resume content, rewrite the resume content to:
            - Match the job requirement more strongly
            - Use modern, professional, and natural-sounding language
            - Keep it in bullet point format
            - Include relevant links if they add value

            Job Description:
            {jd_chunk}

            Original Resume Bullet:
            {resume_chunk}

            {links_str}

            Return only the rewritten bullet.
            """

In [44]:
# === Rewrite function ===
def rewrite_resume_matches(matches, rate_limit=15):
    rewritten_chunks = []
    call_count = 0

    for i, match_group in enumerate(matches):
        jd_chunk = match_group["jd_chunk"]
        for match in match_group["top_resume_matches"]:
            if call_count >= rate_limit:
                print("⚠️ Rate limit reached. Sleeping for 60 seconds...")
                time.sleep(60)
                call_count = 0

            prompt = build_prompt(jd_chunk, match["resume_chunk"], match.get("links", []))
            try:
                response = model_rewrite.generate_content(prompt)
                rewritten = response.text.strip()
            except Exception as e:
                print(f"❌ Error rewriting chunk: {e}")
                rewritten = match["resume_chunk"]

            rewritten_chunks.append({
                "original": match["resume_chunk"],
                "rewritten": rewritten,
                "jd_context": jd_chunk,
                "similarity": match["similarity"],
                "links": match.get("links", [])
            })

            call_count += 1

    return rewritten_chunks

In [47]:
# === MAIN Runner ===
def match_resume_to_jd():
    resume_docs, jd_docs, resume_vectors, jd_vectors = load_vectorstores()
    sim_matrix = compute_similarity_matrix(resume_vectors, jd_vectors)
    matches = rank_resume_matches(jd_docs, resume_docs, sim_matrix, top_k=3)
    #print_matches(matches)
    print("✍️ Rewriting best-matched resume chunks...")
    rewritten = rewrite_resume_matches(matches)

    for item in rewritten:
        print("\n---------------------------")
        print(f"🔹 JD Context:\n{item['jd_context']}")
        print(f"🔸 Original:\n{item['original']}")
        print(f"✅ Rewritten:\n{item['rewritten']}")
        if item["links"]:
            print(f"🔗 Links: {item['links']}")

if __name__ == "__main__":
    match_resume_to_jd()

📥 Loading vectorstores...
🧠 Computing cosine similarity...
✍️ Rewriting best-matched resume chunks...

---------------------------
🔹 JD Context:
AI Engineer
    Enterprise AI/ML Organization
    Reports to Leader of AI Engineering Group

    OVERVIEW
    We are seeking a highly skilled and forward-thinking AI Engineer to join our AI Engineering team. This role is ideal for a hands-on technologist with deep expertise in machine learning (ML) and deep learning (DL), and a strong working knowledge of Generative AI technologies. You will design, build, and optimize intelligent systems that power automation, personalization, and decision-making across our FinTech platforms. This is a unique opportunity to work at the intersection of cutting-edge AI research and real-world enterprise applications.
🔸 Original:
• Implemented a Generative AI model with prompt engineering to assess document grammar, structure, and identify missing information
in paragraphs.
Tech Fortune Technologies Jul 2019 - A